In [1]:
import json
import torch

In [3]:
from MyDataset import MyDataset
from torch.utils.data import DataLoader

In [ ]:
lama_embedding_path = '../llama_embedding/tumor_med_finetuned/dataset_tensor/'
bert_embedding_path = '../bert_embedding/tumor_finetuned/dataset_tensor/'
roberta_embedding_path = '../roberta_embedding/tumor_finetuned/dataset_tensor/'

In [5]:
train_data = MyDataset('train', lama_embedding_path, bert_embedding_path, roberta_embedding_path)
test_data =MyDataset('test', lama_embedding_path, bert_embedding_path, roberta_embedding_path)

In [6]:
train_data = DataLoader(train_data, batch_size=1, shuffle=False)
test_data = DataLoader(test_data, batch_size=1, shuffle=False) 

In [7]:
for i, data in enumerate(train_data):
    print(data)
    break

[tensor([[[ 1.3877, -2.6445,  0.7119,  ...,  0.4238,  0.4985, -0.6724],
         [ 0.3362, -0.6162,  0.2322,  ..., -0.0073, -0.1953, -0.5059],
         [ 0.1300, -0.5361,  0.2517,  ...,  0.2153, -0.0950, -0.3718],
         [ 0.0030, -0.4092,  0.0399,  ...,  0.1614, -0.1187, -0.3621],
         [-0.0638, -0.3369,  0.0263,  ...,  0.2522, -0.1632, -0.2871]]],
       dtype=torch.float16), tensor([[-0.0811, -0.7083,  0.8326,  ..., -0.6350,  0.9180,  0.5636]]), tensor([[-0.5352,  0.6397, -0.1189,  ..., -1.5615, -1.9301, -1.0474]]), tensor([3])]


In [8]:
l_rep, b_rep, r_rep, label = data

In [9]:
b_rep.squeeze().shape

torch.Size([1024])

In [10]:
l_rep.squeeze().shape, b_rep.squeeze().shape, r_rep.squeeze().shape, label.item()

(torch.Size([5, 4096]), torch.Size([1024]), torch.Size([1024]), 3)

In [11]:
# reps 별 코사인 유사도 평균 구해보기
def get_l1_distance(x1, x2):
    return ((x1 - x2).abs()).sum()
def get_l2_distance(x1, x2):
    return ((x1 - x2)**2).sum()**.5
def get_cosine_similarity(x1, x2):
    return (x1 * x2).sum() / ((x1**2).sum()**.5 * (x2**2).sum()**.5)

In [12]:
#get_cosine_similarity(reps_dict[0]['lama'][0].flatten() , reps_dict[0]['lama'][1].flatten()).item()

# Label 끼리의 코사인 유사도
## Train Data

In [13]:
reps_dict = {i : {'lama' : [],
                  'bert' : [],
                'roberta' : [] } for i in range(6)}
cnt = 0
for data in train_data:
    cnt += 1
    l_rep, b_rep, r_rep, label = data
    l_rep = l_rep.squeeze()
    b_rep = b_rep.squeeze()
    r_rep = r_rep.squeeze()
    label = label.item()
    reps_dict[label]['lama'].append(l_rep)
    reps_dict[label]['bert'].append(b_rep)
    reps_dict[label]['roberta'].append(r_rep)
cnt == len(train_data)

True

In [14]:
all_dict = {i :{'lama' : [] ,'bert': [],'roberta': []} for i in range(6)}
for idx in range(6):
    # lama, bert, roberta 각각의 평균 코사인 유사도 구하기
    lama_cosine = []
    bert_cosine = []
    roberta_cosine = []
    for i in range(len(reps_dict[idx]['lama'])):
        for j in range(i+1, len(reps_dict[idx]['lama'])):
            lama_cosine.append(get_cosine_similarity(reps_dict[idx]['lama'][i].flatten() , reps_dict[idx]['lama'][j].flatten()).item())
            bert_cosine.append(get_cosine_similarity(reps_dict[idx]['bert'][i].flatten() , reps_dict[idx]['bert'][j].flatten()).item())
            roberta_cosine.append(get_cosine_similarity(reps_dict[idx]['roberta'][i].flatten() , reps_dict[idx]['roberta'][j].flatten()).item())

    all_dict[idx]['lama'] = torch.tensor(lama_cosine)
    all_dict[idx]['bert'] = torch.tensor(bert_cosine)
    all_dict[idx]['roberta'] = torch.tensor(roberta_cosine)

In [15]:
for i in range(6):
    print(f'Label {i} :')
    print(f'Lama mean : {all_dict[i]["lama"].mean()}, std : {all_dict[i]["lama"].std()}, min : {all_dict[i]["lama"].min()}, max : {all_dict[i]["lama"].max()}')
    print(f'Bert mean : {all_dict[i]["bert"].mean()}, std : {all_dict[i]["bert"].std()}, min : {all_dict[i]["bert"].min()}, max : {all_dict[i]["bert"].max()}')
    print(f'Roberta mean : {all_dict[i]["roberta"].mean()}, std : {all_dict[i]["roberta"].std()}, min : {all_dict[i]["roberta"].min()}, max : {all_dict[i]["roberta"].max()}')
    print()

Label 0 :
Lama mean : 0.9099491238594055, std : 0.031666602939367294, min : 0.75048828125, max : 0.998046875
Bert mean : 0.9927564263343811, std : 0.00888011883944273, min : 0.9261558651924133, max : 0.9999865889549255
Roberta mean : 0.9656540751457214, std : 0.08705567568540573, min : 0.37397775053977966, max : 0.9999938011169434

Label 1 :
Lama mean : 0.8911696076393127, std : 0.042702287435531616, min : 0.7421875, max : 0.9990234375
Bert mean : 0.9966709017753601, std : 0.0028710165061056614, min : 0.9811343550682068, max : 0.9999918937683105
Roberta mean : 0.9827726483345032, std : 0.014596483670175076, min : 0.8483975529670715, max : 0.9997824430465698

Label 2 :
Lama mean : 0.8935725092887878, std : 0.06473805755376816, min : 0.54150390625, max : 0.9951171875
Bert mean : 0.9953146576881409, std : 0.008820051327347755, min : 0.9241839647293091, max : 1.0000001192092896
Roberta mean : 0.9897527098655701, std : 0.0062699466943740845, min : 0.9594651460647583, max : 0.999876797199249

## Test Data

In [16]:
reps_dict = {i : {'lama' : [],
                  'bert' : [],
                'roberta' : [] } for i in range(6)}
cnt = 0
for data in test_data:
    cnt += 1
    l_rep, b_rep, r_rep, label = data
    l_rep = l_rep.squeeze()
    b_rep = b_rep.squeeze()
    r_rep = r_rep.squeeze()
    label = label.item()
    reps_dict[label]['lama'].append(l_rep)
    reps_dict[label]['bert'].append(b_rep)
    reps_dict[label]['roberta'].append(r_rep)
cnt == len(test_data)

True

In [17]:
all_dict = {i :{'lama' : [] ,'bert': [],'roberta': []} for i in range(6)}
for idx in range(6):
    # lama, bert, roberta 각각의 평균 코사인 유사도 구하기
    lama_cosine = []
    bert_cosine = []
    roberta_cosine = []
    for i in range(len(reps_dict[idx]['lama'])):
        for j in range(i+1, len(reps_dict[idx]['lama'])):
            lama_cosine.append(get_cosine_similarity(reps_dict[idx]['lama'][i].flatten() , reps_dict[idx]['lama'][j].flatten()).item())
            bert_cosine.append(get_cosine_similarity(reps_dict[idx]['bert'][i].flatten() , reps_dict[idx]['bert'][j].flatten()).item())
            roberta_cosine.append(get_cosine_similarity(reps_dict[idx]['roberta'][i].flatten() , reps_dict[idx]['roberta'][j].flatten()).item())

    all_dict[idx]['lama'] = torch.tensor(lama_cosine)
    all_dict[idx]['bert'] = torch.tensor(bert_cosine)
    all_dict[idx]['roberta'] = torch.tensor(roberta_cosine)

In [18]:
print('--------------Test Data-------------------')
for i in range(6):
    print(f'Label {i} :')
    print(f'Lama mean : {all_dict[i]["lama"].mean()}, std : {all_dict[i]["lama"].std()}, min : {all_dict[i]["lama"].min()}, max : {all_dict[i]["lama"].max()}')
    print(f'Bert mean : {all_dict[i]["bert"].mean()}, std : {all_dict[i]["bert"].std()}, min : {all_dict[i]["bert"].min()}, max : {all_dict[i]["bert"].max()}')
    print(f'Roberta mean : {all_dict[i]["roberta"].mean()}, std : {all_dict[i]["roberta"].std()}, min : {all_dict[i]["roberta"].min()}, max : {all_dict[i]["roberta"].max()}')
    print()

--------------Test Data-------------------
Label 0 :
Lama mean : 0.8884286880493164, std : 0.04591073468327522, min : 0.72607421875, max : 0.9990234375
Bert mean : 0.8825313448905945, std : 0.3595038950443268, min : -0.2971953749656677, max : 0.9997621774673462
Roberta mean : 0.9556592106819153, std : 0.05133352428674698, min : 0.7699362635612488, max : 0.999946117401123

Label 1 :
Lama mean : 0.8690539598464966, std : 0.06044686213135719, min : 0.68994140625, max : 0.9990234375
Bert mean : 0.6866618394851685, std : 0.4628823399543762, min : -0.3245903551578522, max : 0.999981701374054
Roberta mean : 0.6968095302581787, std : 0.39791131019592285, min : -0.018198715522885323, max : 0.9996711015701294

Label 2 :
Lama mean : 0.8845657110214233, std : 0.053146786987781525, min : 0.76025390625, max : 0.99609375
Bert mean : 0.9619156122207642, std : 0.06619256734848022, min : 0.7581995129585266, max : 0.9999444484710693
Roberta mean : 0.984171986579895, std : 0.021882247179746628, min : 0.89

# 다른 라벨들과의 코사인 유사도 비교
## TrainData

In [19]:
reps_dict = {i : {'lama' : [],
                  'bert' : [],
                'roberta' : [] } for i in range(6)}
cnt = 0
for data in train_data:
    cnt += 1
    l_rep, b_rep, r_rep, label = data
    l_rep = l_rep.squeeze()
    b_rep = b_rep.squeeze()
    r_rep = r_rep.squeeze()
    label = label.item()
    reps_dict[label]['lama'].append(l_rep)
    reps_dict[label]['bert'].append(b_rep)
    reps_dict[label]['roberta'].append(r_rep)
cnt == len(train_data)

True

In [20]:
all_dict = {i : {j :{'lama' : [] ,'bert': [],'roberta': []}  for j in range(6) if i != j } for i in range(6)}
for i in range(6):
    # lama, bert, roberta 각각의 평균 코사인 유사도 구하기

    for j in range(6):
        lama_cosine = []
        bert_cosine = []
        roberta_cosine = []
        if i != j :
            for l in range(len(reps_dict[i]['lama'])):
                for m in range(len(reps_dict[j]['lama'])):
                    lama_cosine.append(get_cosine_similarity(reps_dict[i]['lama'][l].flatten() , reps_dict[j]['lama'][m].flatten()).item())
                    bert_cosine.append(get_cosine_similarity(reps_dict[i]['bert'][l].flatten() , reps_dict[j]['bert'][m].flatten()).item())
                    roberta_cosine.append(get_cosine_similarity(reps_dict[i]['roberta'][l].flatten() , reps_dict[j]['roberta'][m].flatten()).item())
            
            all_dict[i][j]['lama'] = torch.tensor(lama_cosine)
            all_dict[i][j]['bert'] = torch.tensor(bert_cosine)
            all_dict[i][j]['roberta'] = torch.tensor(roberta_cosine)
        else:
            continue

In [21]:
# i, j 레이블 간의 평균 코사인 유사도 구하기
for i in range(6):
    for j in range(6):
        if i != j:
            print(f'Label {i} - Label {j} :')
            print(f'Lama mean : {all_dict[i][j]["lama"].mean()}, std : {all_dict[i][j]["lama"].std()}, min : {all_dict[i][j]["lama"].min()}, max : {all_dict[i][j]["lama"].max()}')
            print(f'Bert mean : {all_dict[i][j]["bert"].mean()}, std : {all_dict[i][j]["bert"].std()}, min : {all_dict[i][j]["bert"].min()}, max : {all_dict[i][j]["bert"].max()}')
            print(f'Roberta mean : {all_dict[i][j]["roberta"].mean()}, std : {all_dict[i][j]["roberta"].std()}, min : {all_dict[i][j]["roberta"].min()}, max : {all_dict[i][j]["roberta"].max()}')
            print()

Label 0 - Label 1 :
Lama mean : 0.8921552896499634, std : 0.03772232308983803, min : 0.72802734375, max : 0.9716796875
Bert mean : -0.1666981279850006, std : 0.010526364669203758, min : -0.208282470703125, max : -0.12093579024076462
Roberta mean : -0.03852460905909538, std : 0.03408439829945564, min : -0.16758498549461365, max : 0.1900651603937149

Label 0 - Label 2 :
Lama mean : 0.8926393389701843, std : 0.05133568122982979, min : 0.556640625, max : 0.97314453125
Bert mean : -0.2327321171760559, std : 0.0077667078003287315, min : -0.2909048795700073, max : -0.19608166813850403
Roberta mean : -0.018670227378606796, std : 0.0872267335653305, min : -0.09566497057676315, max : 0.8220071792602539

Label 0 - Label 3 :
Lama mean : 0.9025869369506836, std : 0.04167019948363304, min : 0.6494140625, max : 0.9697265625
Bert mean : -0.18996699154376984, std : 0.011919560842216015, min : -0.24039670825004578, max : -0.13743461668491364
Roberta mean : -0.03633812814950943, std : 0.05369073525071144

## Test  Data

In [22]:
reps_dict = {i : {'lama' : [],
                  'bert' : [],
                'roberta' : [] } for i in range(6)}
cnt = 0
for data in test_data:
    cnt += 1
    l_rep, b_rep, r_rep, label = data
    l_rep = l_rep.squeeze()
    b_rep = b_rep.squeeze()
    r_rep = r_rep.squeeze()
    label = label.item()
    reps_dict[label]['lama'].append(l_rep)
    reps_dict[label]['bert'].append(b_rep)
    reps_dict[label]['roberta'].append(r_rep)
cnt == len(test_data)

True

In [23]:
all_dict = {i : {j :{'lama' : [] ,'bert': [],'roberta': []}  for j in range(6) if i != j } for i in range(6)}
for i in range(6):
    # lama, bert, roberta 각각의 평균 코사인 유사도 구하기

    for j in range(6):
        lama_cosine = []
        bert_cosine = []
        roberta_cosine = []
        if i != j :
            for l in range(len(reps_dict[i]['lama'])):
                for m in range(len(reps_dict[j]['lama'])):
                    lama_cosine.append(get_cosine_similarity(reps_dict[i]['lama'][l].flatten() , reps_dict[j]['lama'][m].flatten()).item())
                    bert_cosine.append(get_cosine_similarity(reps_dict[i]['bert'][l].flatten() , reps_dict[j]['bert'][m].flatten()).item())
                    roberta_cosine.append(get_cosine_similarity(reps_dict[i]['roberta'][l].flatten() , reps_dict[j]['roberta'][m].flatten()).item())
            
            all_dict[i][j]['lama'] = torch.tensor(lama_cosine)
            all_dict[i][j]['bert'] = torch.tensor(bert_cosine)
            all_dict[i][j]['roberta'] = torch.tensor(roberta_cosine)
        else:
            continue

In [24]:
# i, j 레이블 간의 평균 코사인 유사도 구하기
for i in range(6):
    for j in range(6):
        if i != j:
            print(f'Label {i} - Label {j} :')
            print(f'Lama mean : {all_dict[i][j]["lama"].mean()}, std : {all_dict[i][j]["lama"].std()}, min : {all_dict[i][j]["lama"].min()}, max : {all_dict[i][j]["lama"].max()}')
            print(f'Bert mean : {all_dict[i][j]["bert"].mean()}, std : {all_dict[i][j]["bert"].std()}, min : {all_dict[i][j]["bert"].min()}, max : {all_dict[i][j]["bert"].max()}')
            print(f'Roberta mean : {all_dict[i][j]["roberta"].mean()}, std : {all_dict[i][j]["roberta"].std()}, min : {all_dict[i][j]["roberta"].min()}, max : {all_dict[i][j]["roberta"].max()}')
            print()

Label 0 - Label 1 :
Lama mean : 0.874394953250885, std : 0.049826718866825104, min : 0.6787109375, max : 0.96533203125
Bert mean : -0.14266011118888855, std : 0.13258406519889832, min : -0.2987886369228363, max : 0.9136795997619629
Roberta mean : 0.05507628247141838, std : 0.19220171868801117, min : -0.08155005425214767, max : 0.8289508819580078

Label 0 - Label 2 :
Lama mean : 0.8777032494544983, std : 0.04331820085644722, min : 0.71728515625, max : 0.96142578125
Bert mean : -0.22013546526432037, std : 0.0641045793890953, min : -0.284465491771698, max : 0.1181434914469719
Roberta mean : -0.03148631006479263, std : 0.01759730465710163, min : -0.08271956443786621, max : 0.04119328409433365

Label 0 - Label 3 :
Lama mean : 0.8918478488922119, std : 0.051497478038072586, min : 0.6396484375, max : 0.95263671875
Bert mean : -0.14841324090957642, std : 0.22305595874786377, min : -0.3096223771572113, max : 0.9735559821128845
Roberta mean : 0.008408951573073864, std : 0.11687132716178894, min 